<a href="https://colab.research.google.com/github/adeladd0/take-home-engineering-challenge/blob/main/Take_Home_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Problem
Our San Francisco team loves to eat. They are also a team that loves variety, so they also like to discover new places to eat.

In fact, we have a particular affection for food trucks. One of the great things about Food Trucks in San Francisco is that the city releases a list of them as open data.

Your assignment is to make it possible for us to find a food truck no matter where our work takes us in the city.

This is a freeform assignment. You can write a web API that returns a set of food trucks (our team is fluent in JSON). You can write a web frontend that visualizes the nearby food trucks. We also spend a lot of time in the shell, so a CLI that gives us a couple of local options would be great. And don't be constrained by these ideas if you have a better one!

The only requirement for the assignment is that it give us at least 5 food trucks to choose from a particular latitude and longitude.

Feel free to tackle this problem in a way that demonstrates your expertise of an area -- or takes you out of your comfort zone. For example, if you build Web APIs by day and want to build a frontend to the problem or a completely different language instead, by all means go for it - learning is a core competency in our group. Let us know this context in your solution's documentation.

San Francisco's food truck open dataset is located here and there is an endpoint with a CSV dump of the latest data here. We've included a copy of this data in this repo as well.

Good luck! Please send a link to your solution on Github back to us at least 12 hours before your interview so we can review it before we speak.

In [1]:
#@title Install dependencies
from IPython.display import clear_output
!pip install fastapi nest-asyncio pyngrok uvicorn fsspec
clear_output()

In [2]:
#@title Import Libraries
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import numpy as np
import json,os
from typing import Union , List
from fastapi import FastAPI
from fastapi.testclient import TestClient
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
from os.path import exists

In [3]:
#@title Constants
dataset_path='https://data.sfgov.org/api/views/rqzj-sfat/rows.csv'
pickled_dataset_path="./foodtrucks.pkl"
RESPONSE_CODE_SUCCESS="01"
RESPONSE_CODE_NO_FOOD_TRUCKS="100"
SUCCESS="Successfully retrieved data" 
NO_FOOD_TRUCKS="There are no food trucks at this time. Please try next time"
WELCOME="Welcome to Food truck Service"

In [4]:
#@title Load Dataset
def load_dataset(path=dataset_path):
    try:
        return pd.read_csv(path)
    except:
        print('Could not load dataset')    

def pickle_dataframe(df,file=pickled_dataset_path):
    df.to_pickle(file)
       

def load_pickled_data(file=pickled_dataset_path):
    try:
     return pd.read_pickle(file)
    except (FileNotFoundError, IOError):
        print('Pickled data not found') 

def load_data():
    if exists(pickled_dataset_path):
       return load_pickled_data()
    else:
       df=load_dataset()
       if df:
          pickle_dataframe(df)
       return df 
          

In [5]:
#@title Compute distance between users location and available food trucks
def compute_distance(location,destination=(37.762019,-122.415945)):
    X = [[radians(destination[0]), radians(destination[1])], [radians(location[0]), radians(location[1])]]
    result = haversine_distances(X)
    result * 6371000/1000  # multiply by Earth radius to get kilometers
    distance=np.array(result).item(1)
    #print('distance using sklearn: ', distance)
    return distance    

In [6]:
#@title Get nearest food truck spots 
def get_nearest_food_trucks(latitude,longitude):
    df=load_data()
    if df is not None:
       df['Distance']=df.apply(lambda x: compute_distance(location=(x.Latitude,x.Longitude),destination=(latitude,longitude)), axis=1)
       return df.sort_values(by=['Distance'], ascending=True).head(5)

def get_nearest_food_trucks_as_json(latitude,longitude):
    sorted_df=get_nearest_food_trucks(latitude,longitude)
    if sorted_df is not None:
      json_data = sorted_df.to_json(orient='records')
      response = json.loads(json_data)
      return response    

def display_nearest_food_trucks(latitude,longitude):
    sorted_df=get_nearest_food_trucks(latitude,longitude)
    if sorted_df is not None:
        columns=["locationid","LocationDescription","Address","FoodItems","dayshours","Distance"]
        results=sorted_df[columns]
        results.columns=["locationId","locationDescription","address","foodItems","daysHours","distance"]
        json_data = results.to_json(orient='records')
        return json.loads(json_data)

In [7]:
auth_token = "" #@param {type:"string"}
os.system(f"ngrok authtoken {auth_token}")

0

In [8]:
#@title Response Body
class Request(BaseModel):
    longitude: float
    latitude: float

class Data(BaseModel):
     locationId: Union[int, None] = None
     locationDescription: Union[str, None] = None
     address: Union[str, None] = None
     foodItems : Union[str, None] = None
     daysHours: Union[str, None] = None
     distance: Union[float, None] = None

class FoodTruck(BaseModel):
    responseCode: str
    responseMessage: str
    data:  Union[List[Data], None] = None

In [9]:
#@title Apis to expose endpoints
app = FastAPI()

@app.get("/")
def home():
    return {"response": WELCOME}

@app.post("/foodtrucks/",response_model=FoodTruck)
def load_trucks(request: Request):
    data=display_nearest_food_trucks(request.latitude,request.longitude)
    responseCode=RESPONSE_CODE_SUCCESS if data else RESPONSE_CODE_NO_FOOD_TRUCKS
    responseMessage=SUCCESS if data else NO_FOOD_TRUCKS 
    res={"responseCode": responseCode,"responseMessage": responseMessage,"data":data }
    print("RESPONSE ",res)
    return res        


In [10]:
#@title Test Apis
client = TestClient(app)

def test_home():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"response": WELCOME}

def test_load_trucks():
    response = client.post("/foodtrucks/",
        json={ "longitude": 50.45,"latitude": 0.2045})
    assert response.status_code == 200
    assert response.json()["responseCode"] == RESPONSE_CODE_SUCCESS if response.json()["data"] else RESPONSE_CODE_NO_FOOD_TRUCKS

test_home()
test_load_trucks()    

RESPONSE  {'responseCode': '01', 'responseMessage': 'Successfully retrieved data', 'data': [{'locationId': 1577173, 'locationDescription': 'OTIS ST: GOUGH ST \\ MCCOPPIN ST to 13TH ST \\ DUBOCE AVE \\ HWY 101 NORTHBOUND RAMP \\ MISSION ST (100 - 199)', 'address': '170 OTIS ST', 'foodItems': 'Cold Truck: Breakfast: Sandwiches: Salads: Pre-Packaged Snacks: Beverages', 'daysHours': None, 'distance': 0.8805238676}, {'locationId': 1343454, 'locationDescription': 'HOOPER ST: 07TH ST to 08TH ST (100 - 199)', 'address': 'Assessors Block /Lot', 'foodItems': 'tacos: burritos: quesadilla: tortas sodas', 'daysHours': None, 'distance': 0.8805238676}, {'locationId': 1337457, 'locationDescription': 'OFARRELL ST: BRODERICK ST to SAINT JOSEPHS AVE (2200 - 2299)', 'address': '2200 OFARRELL ST', 'foodItems': 'Cold Truck: Pre-packaged sandwiches: snacks: fruit: various beverages', 'daysHours': None, 'distance': 0.8805238676}, {'locationId': 1336743, 'locationDescription': 'ALANA WAY: COUNTY LINE intersect

In [11]:
#@title Serve Api
ngrok_tunnel= ngrok.connect(8000, port='8000', bind_tls=True)
public_url=ngrok_tunnel.public_url
nest_asyncio.apply()
print('Public URL:', ngrok_tunnel)
print('Api documentation URL:', public_url+"/redoc")
print('Swagger URL:', public_url+"/docs")

Public URL: NgrokTunnel: "https://0f11-34-90-71-4.ngrok.io" -> "http://localhost:8000"
Api documentation URL: https://0f11-34-90-71-4.ngrok.io/redoc
Swagger URL: https://0f11-34-90-71-4.ngrok.io/docs


In [ ]:
uvicorn.run(app,port=8000)

In [12]:
#ngrok.disconnect(public_url=public_url)